In [1]:
import pandas as pd

In [2]:
news_dataset = pd.read_csv('train.csv')

In [3]:
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
news_dataset = news_dataset.fillna('')

In [6]:
news_dataset['content'] = news_dataset['author']+' '+news_dataset['title']

In [7]:
print(news_dataset['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [8]:
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split

In [9]:
port_stem = PorterStemmer()
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [10]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [11]:
print(news_dataset['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [12]:
X = news_dataset['content'].values
y = news_dataset['label'].values

In [13]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [14]:
print(y)

[1 0 1 ... 0 1 1]


In [15]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [16]:
voc_size=10000
corpus=X
print(corpus)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [17]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[1929, 756, 4380, 8665, 3936, 7515, 5013, 386, 9769, 1149, 2618, 2841],
 [7251, 6118, 5705, 5705, 2656, 6352, 8916, 5124, 9203, 439],
 [9845, 7816, 3269, 8469, 3197, 7876],
 [4703, 4110, 2761, 6141, 6015, 4935, 5023, 8868],
 [335, 1167, 9915, 5124, 6588, 8826, 7533, 8980, 5124, 5992, 7898, 2965],
 [7251,
  922,
  9920,
  9588,
  7150,
  374,
  4951,
  8816,
  7847,
  7748,
  1722,
  9326,
  3501,
  2353,
  7257,
  937,
  439],
 [3759, 3759, 8419, 6138, 5864, 1939, 4070, 2725, 8503, 6100, 3170, 9442],
 [6139,
  6118,
  5564,
  3827,
  9751,
  1448,
  8302,
  1945,
  5311,
  3329,
  7439,
  4180,
  9494,
  713],
 [1665, 1658, 2972, 8274, 8816, 3086, 5970, 7469, 8246, 3096, 4180, 9494, 713],
 [3779,
  2949,
  2076,
  3621,
  4782,
  5759,
  1482,
  7852,
  2675,
  6292,
  8816,
  7048,
  4180,
  9494,
  713],
 [9050, 4662, 2050, 3795, 5816, 5587, 515, 3074, 1613, 8059, 8816, 924],
 [1410, 2689, 5801, 1414, 2426, 5588, 4929, 8021, 2846, 9084],
 [6711,
  2927,
  2717,
  6337,
  3201,
  955

In [18]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 1149 2618 2841]
 [   0    0    0 ... 5124 9203  439]
 [   0    0    0 ... 8469 3197 7876]
 ...
 [   0    0    0 ... 4180 9494  713]
 [   0    0    0 ... 2152  900 9618]
 [   0    0    0 ... 9309 3903 1993]]


In [19]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0, 1929,  756, 4380,
       8665, 3936, 7515, 5013,  386, 9769, 1149, 2618, 2841])

In [20]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            400000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 456,501
Trainable params: 456,501
Non-trainable params: 0
_________________________________________________________________
None


In [20]:
len(embedded_docs),y.shape

(20800, (20800,))

In [21]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [22]:
X_final.shape,y_final.shape

((20800, 20), (20800,))

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=0)

In [25]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=32)

Epoch 1/10
436/436 [==============================] - 21s 29ms/step - loss: 0.1488 - accuracy: 0.9371 - val_loss: 0.0357 - val_accuracy: 0.9888
Epoch 2/10
436/436 [==============================] - 11s 25ms/step - loss: 0.0140 - accuracy: 0.9963 - val_loss: 0.0357 - val_accuracy: 0.9873
Epoch 3/10
436/436 [==============================] - 11s 25ms/step - loss: 0.0039 - accuracy: 0.9988 - val_loss: 0.0262 - val_accuracy: 0.9911
Epoch 4/10
436/436 [==============================] - 11s 24ms/step - loss: 9.7564e-04 - accuracy: 0.9998 - val_loss: 0.0416 - val_accuracy: 0.9878
Epoch 5/10
436/436 [==============================] - 10s 24ms/step - loss: 0.0015 - accuracy: 0.9996 - val_loss: 0.0454 - val_accuracy: 0.9898
Epoch 6/10
436/436 [==============================] - 12s 28ms/step - loss: 2.8768e-04 - accuracy: 0.9999 - val_loss: 0.0443 - val_accuracy: 0.9901
Epoch 7/10
436/436 [==============================] - 11s 25ms/step - loss: 3.3471e-05 - accuracy: 1.0000 - val_loss: 0.0546 - v

In [26]:
y_pred = model.predict(X_test)
y_pred = np.round(y_pred).astype(int)


215/215 [==============================] - 2s 4ms/step


In [27]:
from sklearn.metrics import confusion_matrix

In [28]:
confusion_matrix(y_test,y_pred)

array([[3364,   26],
       [  51, 3423]], dtype=int64)

In [29]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9887820512820513

In [35]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.6))
model.add(LSTM(100))
model.add(Dropout(0.6))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 20, 40)            400000    
                                                                 
 dropout_6 (Dropout)         (None, 20, 40)            0         
                                                                 
 lstm_3 (LSTM)               (None, 100)               56400     
                                                                 
 dropout_7 (Dropout)         (None, 100)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
Total params: 456,501
Trainable params: 456,501
Non-trainable params: 0
_________________________________________________________________
None


In [25]:
X_final.shape,y_final.shape

((20800, 20), (20800,))

In [36]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
218/218 [==============================] - 19s 54ms/step - loss: 0.2717 - accuracy: 0.8740 - val_loss: 0.0613 - val_accuracy: 0.9811
Epoch 2/10
218/218 [==============================] - 10s 47ms/step - loss: 0.0486 - accuracy: 0.9847 - val_loss: 0.0299 - val_accuracy: 0.9907
Epoch 3/10
218/218 [==============================] - 10s 46ms/step - loss: 0.0231 - accuracy: 0.9939 - val_loss: 0.0252 - val_accuracy: 0.9917
Epoch 4/10
218/218 [==============================] - 11s 49ms/step - loss: 0.0126 - accuracy: 0.9966 - val_loss: 0.0258 - val_accuracy: 0.9918
Epoch 5/10
218/218 [==============================] - 10s 46ms/step - loss: 0.0087 - accuracy: 0.9977 - val_loss: 0.0247 - val_accuracy: 0.9923
Epoch 6/10
218/218 [==============================] - 10s 45ms/step - loss: 0.0080 - accuracy: 0.9981 - val_loss: 0.0340 - val_accuracy: 0.9895
Epoch 7/10
218/218 [==============================] - 10s 45ms/step - loss: 0.0068 - accuracy: 0.9981 - val_loss: 0.0237 - val_accuracy:

In [37]:
y_pred = model.predict(X_test)
y_pred = np.round(y_pred).astype(int)


215/215 [==============================] - 4s 12ms/step


In [28]:
from sklearn.metrics import confusion_matrix

In [39]:
confusion_matrix(y_test,y_pred)

array([[3367,   23],
       [  40, 3434]], dtype=int64)

In [38]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9908216783216783

In [95]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model11=Sequential()
model11.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model11.add(Dropout(0.5))
model11.add(LSTM(100,return_sequences=True))
model11.add(Dropout(0.5))
model11.add(LSTM(100,return_sequences=False))
model11.add(Dense(1,activation='sigmoid'))
model11.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_14 (Embedding)    (None, 20, 40)            400000    
                                                                 
 dropout_27 (Dropout)        (None, 20, 40)            0         
                                                                 
 lstm_9 (LSTM)               (None, 20, 100)           56400     
                                                                 
 dropout_28 (Dropout)        (None, 20, 100)           0         
                                                                 
 lstm_10 (LSTM)              (None, 100)               80400     
                                                                 
 dense_14 (Dense)            (None, 1)                 101       
                                                                 
Total params: 536,901
Trainable params: 536,901
Non-t

In [96]:
model11.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
218/218 [==============================] - 34s 101ms/step - loss: 0.2342 - accuracy: 0.8911 - val_loss: 0.0457 - val_accuracy: 0.9875
Epoch 2/10
218/218 [==============================] - 20s 92ms/step - loss: 0.0321 - accuracy: 0.9908 - val_loss: 0.0385 - val_accuracy: 0.9898
Epoch 3/10
218/218 [==============================] - 20s 92ms/step - loss: 0.0151 - accuracy: 0.9960 - val_loss: 0.0439 - val_accuracy: 0.9859
Epoch 4/10
218/218 [==============================] - 20s 90ms/step - loss: 0.0120 - accuracy: 0.9971 - val_loss: 0.0328 - val_accuracy: 0.9899
Epoch 5/10
218/218 [==============================] - 20s 92ms/step - loss: 0.0076 - accuracy: 0.9977 - val_loss: 0.0269 - val_accuracy: 0.9921
Epoch 6/10
218/218 [==============================] - 20s 93ms/step - loss: 0.0073 - accuracy: 0.9976 - val_loss: 0.0298 - val_accuracy: 0.9917
Epoch 7/10
218/218 [==============================] - 20s 92ms/step - loss: 0.0023 - accuracy: 0.9994 - val_loss: 0.0325 - val_accuracy

In [97]:
y_pred11 = model11.predict(X_test)
y_pred11 = np.round(y_pred11).astype(int)

215/215 [==============================] - 8s 20ms/step


In [98]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred11)

array([[3375,   15],
       [  25, 3449]], dtype=int64)

In [99]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred11)

0.9941724941724942

In [46]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=60
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(120))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [47]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=32)

Epoch 1/10
436/436 [==============================] - 17s 32ms/step - loss: 0.1528 - accuracy: 0.9367 - val_loss: 0.0424 - val_accuracy: 0.9853
Epoch 2/10
436/436 [==============================] - 16s 37ms/step - loss: 0.0186 - accuracy: 0.9951 - val_loss: 0.0669 - val_accuracy: 0.9757
Epoch 3/10
436/436 [==============================] - 16s 37ms/step - loss: 0.0084 - accuracy: 0.9983 - val_loss: 0.0364 - val_accuracy: 0.9902
Epoch 4/10
436/436 [==============================] - 15s 34ms/step - loss: 0.0038 - accuracy: 0.9987 - val_loss: 0.0365 - val_accuracy: 0.9911
Epoch 5/10
436/436 [==============================] - 15s 35ms/step - loss: 0.0021 - accuracy: 0.9994 - val_loss: 0.0638 - val_accuracy: 0.9883
Epoch 6/10
436/436 [==============================] - 18s 41ms/step - loss: 0.0033 - accuracy: 0.9990 - val_loss: 0.0417 - val_accuracy: 0.9911
Epoch 7/10
436/436 [==============================] - 19s 45ms/step - loss: 8.2023e-04 - accuracy: 0.9998 - val_loss: 0.0395 - val_accur

In [48]:
y_pred = model.predict(X_test)
y_pred = np.round(y_pred).astype(int)

215/215 [==============================] - 5s 17ms/step


In [49]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[3359,   31],
       [  27, 3447]], dtype=int64)

In [50]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9915501165501166

In [56]:
## Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Dropout(0.1))
model1.add(Bidirectional(LSTM(150)))
model1.add(Dropout(0.1))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 20, 40)            400000    
                                                                 
 dropout_15 (Dropout)        (None, 20, 40)            0         
                                                                 
 bidirectional_4 (Bidirectio  (None, 300)              229200    
 nal)                                                            
                                                                 
 dropout_16 (Dropout)        (None, 300)               0         
                                                                 
 dense_8 (Dense)             (None, 1)                 301       
                                                                 
Total params: 629,501
Trainable params: 629,501
Non-trainable params: 0
________________________________________________

In [57]:
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
218/218 [==============================] - 32s 93ms/step - loss: 0.1695 - accuracy: 0.9264 - val_loss: 0.0458 - val_accuracy: 0.9841
Epoch 2/10
218/218 [==============================] - 19s 87ms/step - loss: 0.0205 - accuracy: 0.9945 - val_loss: 0.0409 - val_accuracy: 0.9862
Epoch 3/10
218/218 [==============================] - 19s 85ms/step - loss: 0.0087 - accuracy: 0.9976 - val_loss: 0.0423 - val_accuracy: 0.9902
Epoch 4/10
218/218 [==============================] - 20s 93ms/step - loss: 0.0047 - accuracy: 0.9987 - val_loss: 0.0524 - val_accuracy: 0.9866
Epoch 5/10
218/218 [==============================] - 20s 90ms/step - loss: 0.0019 - accuracy: 0.9992 - val_loss: 0.0484 - val_accuracy: 0.9894
Epoch 6/10
218/218 [==============================] - 19s 87ms/step - loss: 0.0025 - accuracy: 0.9991 - val_loss: 0.0631 - val_accuracy: 0.9854
Epoch 7/10
218/218 [==============================] - 19s 88ms/step - loss: 0.0039 - accuracy: 0.9988 - val_loss: 0.0474 - val_accuracy:

In [58]:
y_pred1 = model1.predict(X_test)
y_pred1 = np.round(y_pred1).astype(int)

215/215 [==============================] - 8s 16ms/step


In [60]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred1)

array([[3366,   24],
       [  33, 3441]], dtype=int64)

In [59]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1)

0.9916958041958042

In [106]:
embedding_vector_features=40
model12=Sequential()
model12.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model12.add(Dropout(0.6))
model12.add(Bidirectional(LSTM(100,return_sequences=True)))
model12.add(Dropout(0.6))
model12.add(Bidirectional(LSTM(100,return_sequences=False)))
model12.add(Dense(1,activation='sigmoid'))
model12.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_19 (Embedding)    (None, 20, 40)            400000    
                                                                 
 dropout_37 (Dropout)        (None, 20, 40)            0         
                                                                 
 bidirectional_5 (Bidirectio  (None, 20, 200)          112800    
 nal)                                                            
                                                                 
 dropout_38 (Dropout)        (None, 20, 200)           0         
                                                                 
 bidirectional_6 (Bidirectio  (None, 200)              240800    
 nal)                                                            
                                                                 
 dense_19 (Dense)            (None, 1)               

In [107]:
model12.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
218/218 [==============================] - 55s 169ms/step - loss: 0.1874 - accuracy: 0.9181 - val_loss: 0.0445 - val_accuracy: 0.9879
Epoch 2/10
218/218 [==============================] - 33s 153ms/step - loss: 0.0372 - accuracy: 0.9885 - val_loss: 0.0282 - val_accuracy: 0.9910
Epoch 3/10
218/218 [==============================] - 34s 154ms/step - loss: 0.0213 - accuracy: 0.9935 - val_loss: 0.0256 - val_accuracy: 0.9924
Epoch 4/10
218/218 [==============================] - 33s 152ms/step - loss: 0.0112 - accuracy: 0.9963 - val_loss: 0.0349 - val_accuracy: 0.9885
Epoch 5/10
218/218 [==============================] - 33s 153ms/step - loss: 0.0117 - accuracy: 0.9958 - val_loss: 0.0256 - val_accuracy: 0.9926
Epoch 6/10
218/218 [==============================] - 33s 153ms/step - loss: 0.0059 - accuracy: 0.9981 - val_loss: 0.0346 - val_accuracy: 0.9924
Epoch 7/10
218/218 [==============================] - 33s 153ms/step - loss: 0.0072 - accuracy: 0.9980 - val_loss: 0.0319 - val_ac

In [108]:
y_pred12 = model12.predict(X_test)
y_pred12 = np.round(y_pred12).astype(int)

215/215 [==============================] - 10s 24ms/step


In [110]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred12)

array([[3376,   14],
       [  46, 3428]], dtype=int64)

In [109]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred12)

0.9912587412587412

In [113]:
from tensorflow.keras.layers import GRU
embedding_vector_features=40
model51=Sequential()
model51.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model51.add(Dropout(0.3))
model51.add(GRU(300,return_sequences=True))
model51.add(Dropout(0.3))
model51.add(GRU(300,return_sequences=False))
model51.add(Dense(1,activation='sigmoid'))
model51.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])
print(model51.summary())

Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_23 (Embedding)    (None, 20, 40)            400000    
                                                                 
 dropout_45 (Dropout)        (None, 20, 40)            0         
                                                                 
 gru_7 (GRU)                 (None, 20, 300)           307800    
                                                                 
 dropout_46 (Dropout)        (None, 20, 300)           0         
                                                                 
 gru_8 (GRU)                 (None, 300)               541800    
                                                                 
 dense_23 (Dense)            (None, 1)                 301       
                                                                 
Total params: 1,249,901
Trainable params: 1,249,901
N

In [114]:
model51.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
218/218 [==============================] - 82s 313ms/step - loss: 0.1680 - accuracy: 0.9237 - val_loss: 0.0432 - val_accuracy: 0.9853
Epoch 2/10
218/218 [==============================] - 64s 295ms/step - loss: 0.0158 - accuracy: 0.9953 - val_loss: 0.0241 - val_accuracy: 0.9934
Epoch 3/10
218/218 [==============================] - 66s 305ms/step - loss: 0.0062 - accuracy: 0.9983 - val_loss: 0.0514 - val_accuracy: 0.9881
Epoch 4/10
218/218 [==============================] - 62s 285ms/step - loss: 0.0037 - accuracy: 0.9988 - val_loss: 0.0362 - val_accuracy: 0.9910
Epoch 5/10
218/218 [==============================] - 64s 293ms/step - loss: 0.0018 - accuracy: 0.9992 - val_loss: 0.0395 - val_accuracy: 0.9905
Epoch 6/10
218/218 [==============================] - 68s 310ms/step - loss: 0.0033 - accuracy: 0.9990 - val_loss: 0.0401 - val_accuracy: 0.9924
Epoch 7/10
218/218 [==============================] - 65s 297ms/step - loss: 0.0038 - accuracy: 0.9992 - val_loss: 0.0347 - val_ac

In [115]:
y_pred51 = model51.predict(X_test)
y_pred51 = np.round(y_pred51).astype(int)

215/215 [==============================] - 18s 78ms/step


In [116]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred51)

0.9925699300699301

In [117]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred51)

array([[3377,   13],
       [  38, 3436]], dtype=int64)

In [79]:
from tensorflow.keras.layers import GRU
embedding_vector_features=40
model5=Sequential()
model5.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model5.add(Dropout(0.4))
model5.add(GRU(200))
model5.add(Dropout(0.4))
model5.add(Dense(1,activation='sigmoid'))
model5.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])
print(model5.summary())

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_13 (Embedding)    (None, 20, 40)            400000    
                                                                 
 dropout_25 (Dropout)        (None, 20, 40)            0         
                                                                 
 gru_4 (GRU)                 (None, 200)               145200    
                                                                 
 dropout_26 (Dropout)        (None, 200)               0         
                                                                 
 dense_13 (Dense)            (None, 1)                 201       
                                                                 
Total params: 545,401
Trainable params: 545,401
Non-trainable params: 0
_________________________________________________________________
None


In [80]:
model5.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
218/218 [==============================] - 23s 69ms/step - loss: 0.1957 - accuracy: 0.9211 - val_loss: 0.0461 - val_accuracy: 0.9883
Epoch 2/10
218/218 [==============================] - 14s 63ms/step - loss: 0.0247 - accuracy: 0.9928 - val_loss: 0.0295 - val_accuracy: 0.9901
Epoch 3/10
218/218 [==============================] - 14s 63ms/step - loss: 0.0092 - accuracy: 0.9974 - val_loss: 0.0253 - val_accuracy: 0.9920
Epoch 4/10
218/218 [==============================] - 14s 65ms/step - loss: 0.0087 - accuracy: 0.9976 - val_loss: 0.0379 - val_accuracy: 0.9878
Epoch 5/10
218/218 [==============================] - 14s 63ms/step - loss: 0.0041 - accuracy: 0.9988 - val_loss: 0.0235 - val_accuracy: 0.9934
Epoch 6/10
218/218 [==============================] - 14s 63ms/step - loss: 0.0024 - accuracy: 0.9994 - val_loss: 0.0262 - val_accuracy: 0.9929
Epoch 7/10
218/218 [==============================] - 14s 63ms/step - loss: 0.0020 - accuracy: 0.9996 - val_loss: 0.0321 - val_accuracy:

In [81]:
y_pred5 = model5.predict(X_test)
y_pred5 = np.round(y_pred5).astype(int)

215/215 [==============================] - 5s 14ms/step


In [82]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred5)

0.993006993006993

In [65]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred5)

array([[3372,   18],
       [  17, 3457]], dtype=int64)

In [72]:
from keras.layers import Conv1D, MaxPooling1D
embedding_vector_features=20
model2=Sequential()
model2.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model2.add(Conv1D(200, kernel_size=3, activation = "relu"))
model2.add(MaxPooling1D())
model2.add(Dropout(0.4))
model2.add(LSTM(100,return_sequences=True)) 
model2.add(Dropout(0.4))
model2.add(LSTM(100,return_sequences=False)) 
model2.add(Dense(1,activation='sigmoid'))
model2.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model2.summary())

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (None, 20, 20)            200000    
                                                                 
 conv1d_10 (Conv1D)          (None, 18, 200)           12200     
                                                                 
 max_pooling1d_10 (MaxPoolin  (None, 9, 200)           0         
 g1D)                                                            
                                                                 
 dropout_19 (Dropout)        (None, 9, 200)            0         
                                                                 
 lstm_15 (LSTM)              (None, 9, 100)            120400    
                                                                 
 dropout_20 (Dropout)        (None, 9, 100)            0         
                                                     

In [73]:
model2.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
218/218 [==============================] - 14s 39ms/step - loss: 0.1832 - accuracy: 0.9152 - val_loss: 0.0310 - val_accuracy: 0.9916
Epoch 2/10
218/218 [==============================] - 8s 37ms/step - loss: 0.0154 - accuracy: 0.9959 - val_loss: 0.0257 - val_accuracy: 0.9924
Epoch 3/10
218/218 [==============================] - 8s 37ms/step - loss: 0.0072 - accuracy: 0.9986 - val_loss: 0.0318 - val_accuracy: 0.9902
Epoch 4/10
218/218 [==============================] - 8s 39ms/step - loss: 0.0038 - accuracy: 0.9991 - val_loss: 0.0499 - val_accuracy: 0.9885
Epoch 5/10
218/218 [==============================] - 9s 42ms/step - loss: 0.0055 - accuracy: 0.9981 - val_loss: 0.0307 - val_accuracy: 0.9913
Epoch 6/10
218/218 [==============================] - 9s 43ms/step - loss: 8.8178e-04 - accuracy: 0.9998 - val_loss: 0.0415 - val_accuracy: 0.9898
Epoch 7/10
218/218 [==============================] - 9s 42ms/step - loss: 5.5520e-04 - accuracy: 0.9998 - val_loss: 0.0808 - val_accurac

In [74]:
y_pred2 = model2.predict(X_test)
y_pred2 = np.round(y_pred2).astype(int)

215/215 [==============================] - 4s 7ms/step


In [75]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred2)

0.9934440559440559

In [76]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred2)

array([[3378,   12],
       [  33, 3441]], dtype=int64)

In [120]:
from keras.layers import Conv1D, MaxPooling1D
embedding_vector_features=20
model4=Sequential()
model4.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model4.add(Conv1D(200, kernel_size=3, activation = "relu"))
model4.add(MaxPooling1D())
model4.add(Dropout(0.3))
model4.add(Bidirectional(LSTM(100,return_sequences=True)))
model4.add(Dropout(0.3))
model4.add(Bidirectional(LSTM(100,return_sequences=False)))
model4.add(Dense(1,activation='sigmoid'))
model4.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model4.summary())

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_21 (Embedding)    (None, 20, 20)            200000    
                                                                 
 conv1d_21 (Conv1D)          (None, 18, 200)           12200     
                                                                 
 max_pooling1d_21 (MaxPoolin  (None, 9, 200)           0         
 g1D)                                                            
                                                                 
 dropout_42 (Dropout)        (None, 9, 200)            0         
                                                                 
 bidirectional_15 (Bidirecti  (None, 9, 200)           240800    
 onal)                                                           
                                                                 
 dropout_43 (Dropout)        (None, 9, 200)          

In [121]:
model4.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
218/218 [==============================] - 38s 94ms/step - loss: 0.1594 - accuracy: 0.9249 - val_loss: 0.0327 - val_accuracy: 0.9910
Epoch 2/10
218/218 [==============================] - 17s 78ms/step - loss: 0.0146 - accuracy: 0.9963 - val_loss: 0.0251 - val_accuracy: 0.9927
Epoch 3/10
218/218 [==============================] - 17s 80ms/step - loss: 0.0034 - accuracy: 0.9991 - val_loss: 0.0399 - val_accuracy: 0.9886
Epoch 4/10
218/218 [==============================] - 17s 79ms/step - loss: 0.0024 - accuracy: 0.9991 - val_loss: 0.0362 - val_accuracy: 0.9918
Epoch 5/10
218/218 [==============================] - 17s 78ms/step - loss: 8.2295e-04 - accuracy: 0.9997 - val_loss: 0.0933 - val_accuracy: 0.9828
Epoch 6/10
218/218 [==============================] - 17s 79ms/step - loss: 0.0080 - accuracy: 0.9975 - val_loss: 0.0303 - val_accuracy: 0.9911
Epoch 7/10
218/218 [==============================] - 18s 81ms/step - loss: 0.0024 - accuracy: 0.9993 - val_loss: 0.0346 - val_accur

In [122]:
y_pred4 = model4.predict(X_test)
y_pred4 = np.round(y_pred4).astype(int)

215/215 [==============================] - 8s 14ms/step


In [123]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred4)

0.9919871794871795

In [99]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred4)

array([[3357,   33],
       [  15, 3459]], dtype=int64)

In [124]:
embedding_vector_features=20
model11=Sequential()
model11.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model11.add(Dropout(0.3))
model11.add(Bidirectional(LSTM(100,return_sequences=True)))
model11.add(Dropout(0.3))
model11.add(Bidirectional(LSTM(100,return_sequences=False)))
model11.add(Dense(1,activation='sigmoid'))
model11.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model11.summary())

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_22 (Embedding)    (None, 20, 20)            200000    
                                                                 
 dropout_44 (Dropout)        (None, 20, 20)            0         
                                                                 
 bidirectional_17 (Bidirecti  (None, 20, 200)          96800     
 onal)                                                           
                                                                 
 dropout_45 (Dropout)        (None, 20, 200)           0         
                                                                 
 bidirectional_18 (Bidirecti  (None, 200)              240800    
 onal)                                                           
                                                                 
 dense_22 (Dense)            (None, 1)               

In [125]:
model11.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)


Epoch 1/10
218/218 [==============================] - 55s 150ms/step - loss: 0.1874 - accuracy: 0.9158 - val_loss: 0.0487 - val_accuracy: 0.9831
Epoch 2/10
218/218 [==============================] - 28s 129ms/step - loss: 0.0292 - accuracy: 0.9907 - val_loss: 0.0406 - val_accuracy: 0.9859
Epoch 3/10
218/218 [==============================] - 28s 127ms/step - loss: 0.0164 - accuracy: 0.9948 - val_loss: 0.0438 - val_accuracy: 0.9860
Epoch 4/10
218/218 [==============================] - 27s 124ms/step - loss: 0.0088 - accuracy: 0.9970 - val_loss: 0.0496 - val_accuracy: 0.9863
Epoch 5/10
218/218 [==============================] - 27s 124ms/step - loss: 0.0087 - accuracy: 0.9973 - val_loss: 0.0936 - val_accuracy: 0.9822
Epoch 6/10
218/218 [==============================] - 28s 127ms/step - loss: 0.0053 - accuracy: 0.9985 - val_loss: 0.0407 - val_accuracy: 0.9879
Epoch 7/10
218/218 [==============================] - 27s 126ms/step - loss: 0.0061 - accuracy: 0.9985 - val_loss: 0.0504 - val_ac

In [126]:
y_pred11 = model11.predict(X_test)
y_pred11 = np.round(y_pred11).astype(int)

215/215 [==============================] - 75s 23ms/step


In [127]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred11)


0.9825174825174825

In [128]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred11)

array([[3309,   81],
       [  39, 3435]], dtype=int64)

In [135]:
embedding_vector_features=20
model3=Sequential()
model3.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model3.add(Dropout(0.4))
model3.add(LSTM(100 , return_sequences=True))
model3.add(Dropout(0.4))
model3.add(LSTM(100,return_sequences=False))
model3.add(Dense(1,activation='sigmoid'))
model3.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])
print(model3.summary())

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_24 (Embedding)    (None, 20, 20)            200000    
                                                                 
 dropout_48 (Dropout)        (None, 20, 20)            0         
                                                                 
 lstm_38 (LSTM)              (None, 20, 100)           48400     
                                                                 
 dropout_49 (Dropout)        (None, 20, 100)           0         
                                                                 
 lstm_39 (LSTM)              (None, 100)               80400     
                                                                 
 dense_24 (Dense)            (None, 1)                 101       
                                                                 
Total params: 328,901
Trainable params: 328,901
Non-t

In [136]:
model3.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
218/218 [==============================] - 32s 98ms/step - loss: 0.2434 - accuracy: 0.8894 - val_loss: 0.0540 - val_accuracy: 0.9830
Epoch 2/10
218/218 [==============================] - 19s 88ms/step - loss: 0.0344 - accuracy: 0.9895 - val_loss: 0.0271 - val_accuracy: 0.9910
Epoch 3/10
218/218 [==============================] - 19s 87ms/step - loss: 0.0127 - accuracy: 0.9959 - val_loss: 0.0294 - val_accuracy: 0.9917
Epoch 4/10
218/218 [==============================] - 19s 88ms/step - loss: 0.0083 - accuracy: 0.9976 - val_loss: 0.0286 - val_accuracy: 0.9904
Epoch 5/10
218/218 [==============================] - 19s 88ms/step - loss: 0.0056 - accuracy: 0.9987 - val_loss: 0.0316 - val_accuracy: 0.9901
Epoch 6/10
218/218 [==============================] - 19s 89ms/step - loss: 0.0059 - accuracy: 0.9979 - val_loss: 0.0306 - val_accuracy: 0.9920
Epoch 7/10
218/218 [==============================] - 19s 88ms/step - loss: 0.0033 - accuracy: 0.9989 - val_loss: 0.0419 - val_accuracy:

In [137]:
y_pred3 = model3.predict(X_test)
y_pred3 = np.round(y_pred3).astype(int)

215/215 [==============================] - 9s 25ms/step


In [138]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred3)

0.9912587412587412

In [139]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred3)

array([[3349,   41],
       [  19, 3455]], dtype=int64)

In [133]:
from keras.layers import Conv1D, MaxPooling1D
embedding_vector_features=40
model3=Sequential()
model3.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model3.add(Conv1D(200, kernel_size=3, activation = "relu"))
model3.add(MaxPooling1D())
model3.add(Dropout(0.6))
model3.add(LSTM(100))
model3.add(Dropout(0.6))
model3.add(Dense(1,activation='sigmoid'))
model3.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model3.summary())

Model: "sequential_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_27 (Embedding)    (None, 20, 40)            400000    
                                                                 
 conv1d_3 (Conv1D)           (None, 18, 200)           24200     
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 9, 200)           0         
 1D)                                                             
                                                                 
 dropout_53 (Dropout)        (None, 9, 200)            0         
                                                                 
 lstm_27 (LSTM)              (None, 100)               120400    
                                                                 
 dropout_54 (Dropout)        (None, 100)               0         
                                                     

In [134]:
model3.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
218/218 [==============================] - 34s 75ms/step - loss: 0.1663 - accuracy: 0.9254 - val_loss: 0.0282 - val_accuracy: 0.9916
Epoch 2/10
218/218 [==============================] - 11s 50ms/step - loss: 0.0180 - accuracy: 0.9954 - val_loss: 0.0293 - val_accuracy: 0.9901
Epoch 3/10
218/218 [==============================] - 12s 54ms/step - loss: 0.0074 - accuracy: 0.9982 - val_loss: 0.0311 - val_accuracy: 0.9905
Epoch 4/10
218/218 [==============================] - 14s 64ms/step - loss: 0.0046 - accuracy: 0.9989 - val_loss: 0.0296 - val_accuracy: 0.9911
Epoch 5/10
218/218 [==============================] - 14s 65ms/step - loss: 0.0044 - accuracy: 0.9989 - val_loss: 0.0306 - val_accuracy: 0.9918
Epoch 6/10
218/218 [==============================] - 15s 68ms/step - loss: 0.0036 - accuracy: 0.9989 - val_loss: 0.0318 - val_accuracy: 0.9923
Epoch 7/10
218/218 [==============================] - 13s 61ms/step - loss: 0.0023 - accuracy: 0.9993 - val_loss: 0.0355 - val_accuracy:

In [135]:
y_pred3 = model3.predict(X_test)
y_pred3 = np.round(y_pred3).astype(int)

215/215 [==============================] - 6s 5ms/step


In [136]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred3)

0.9918414918414918

In [137]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred3)

array([[3377,   13],
       [  43, 3431]], dtype=int64)

In [138]:
from keras.layers import Conv1D, MaxPooling1D
embedding_vector_features=40
model6=Sequential()
model6.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model6.add(Conv1D(200, kernel_size=3, activation = "relu"))
model6.add(MaxPooling1D())
model6.add(Dropout(0.6))
model6.add(Bidirectional(LSTM(100)))
model6.add(Dropout(0.6))
model6.add(Dense(1,activation='sigmoid'))
model6.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model6.summary())

Model: "sequential_28"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_28 (Embedding)    (None, 20, 40)            400000    
                                                                 
 conv1d_4 (Conv1D)           (None, 18, 200)           24200     
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 9, 200)           0         
 1D)                                                             
                                                                 
 dropout_55 (Dropout)        (None, 9, 200)            0         
                                                                 
 bidirectional_12 (Bidirecti  (None, 200)              240800    
 onal)                                                           
                                                                 
 dropout_56 (Dropout)        (None, 200)             

In [139]:
model6.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
218/218 [==============================] - 48s 82ms/step - loss: 0.1650 - accuracy: 0.9262 - val_loss: 0.0304 - val_accuracy: 0.9904
Epoch 2/10
218/218 [==============================] - 13s 59ms/step - loss: 0.0182 - accuracy: 0.9956 - val_loss: 0.0311 - val_accuracy: 0.9901
Epoch 3/10
218/218 [==============================] - 11s 52ms/step - loss: 0.0074 - accuracy: 0.9984 - val_loss: 0.0366 - val_accuracy: 0.9886
Epoch 4/10
218/218 [==============================] - 13s 58ms/step - loss: 0.0035 - accuracy: 0.9989 - val_loss: 0.0315 - val_accuracy: 0.9913
Epoch 5/10
218/218 [==============================] - 11s 50ms/step - loss: 0.0044 - accuracy: 0.9986 - val_loss: 0.0373 - val_accuracy: 0.9897
Epoch 6/10
218/218 [==============================] - 13s 60ms/step - loss: 0.0019 - accuracy: 0.9994 - val_loss: 0.0359 - val_accuracy: 0.9908
Epoch 7/10
218/218 [==============================] - 11s 49ms/step - loss: 0.0042 - accuracy: 0.9987 - val_loss: 0.0290 - val_accuracy:

In [140]:
y_pred6 = model6.predict(X_test)
y_pred6 = np.round(y_pred6).astype(int)

215/215 [==============================] - 9s 14ms/step


In [141]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred6)

0.9889277389277389

In [142]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred6)

array([[3378,   12],
       [  64, 3410]], dtype=int64)

In [143]:
from keras.layers import Conv1D, MaxPooling1D
embedding_vector_features=40
model7=Sequential()
model7.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model7.add(Conv1D(200, kernel_size=3, activation = "relu"))
model7.add(MaxPooling1D())
model7.add(Dropout(0.3))
model7.add(GRU(300))
model7.add(Dropout(0.3))
model7.add(Dense(1,activation='sigmoid'))
model7.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model7.summary())

Model: "sequential_29"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_29 (Embedding)    (None, 20, 40)            400000    
                                                                 
 conv1d_5 (Conv1D)           (None, 18, 200)           24200     
                                                                 
 max_pooling1d_5 (MaxPooling  (None, 9, 200)           0         
 1D)                                                             
                                                                 
 dropout_57 (Dropout)        (None, 9, 200)            0         
                                                                 
 gru_10 (GRU)                (None, 300)               451800    
                                                                 
 dropout_58 (Dropout)        (None, 300)               0         
                                                     

In [144]:
model7.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
218/218 [==============================] - 38s 113ms/step - loss: 0.1544 - accuracy: 0.9307 - val_loss: 0.0328 - val_accuracy: 0.9916
Epoch 2/10
218/218 [==============================] - 19s 85ms/step - loss: 0.0103 - accuracy: 0.9974 - val_loss: 0.0229 - val_accuracy: 0.9934
Epoch 3/10
218/218 [==============================] - 18s 84ms/step - loss: 0.0029 - accuracy: 0.9994 - val_loss: 0.0216 - val_accuracy: 0.9948
Epoch 4/10
218/218 [==============================] - 18s 84ms/step - loss: 0.0012 - accuracy: 0.9996 - val_loss: 0.0224 - val_accuracy: 0.9946
Epoch 5/10
218/218 [==============================] - 18s 84ms/step - loss: 1.7726e-04 - accuracy: 1.0000 - val_loss: 0.0279 - val_accuracy: 0.9948
Epoch 6/10
218/218 [==============================] - 18s 85ms/step - loss: 1.0913e-04 - accuracy: 1.0000 - val_loss: 0.0285 - val_accuracy: 0.9946
Epoch 7/10
218/218 [==============================] - 18s 84ms/step - loss: 3.4776e-05 - accuracy: 1.0000 - val_loss: 0.0308 - 

In [145]:
y_pred7 = model7.predict(X_test)
y_pred7 = np.round(y_pred7).astype(int)

215/215 [==============================] - 6s 21ms/step


In [146]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred7)

0.9943181818181818

In [147]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred7)

array([[3375,   15],
       [  24, 3450]], dtype=int64)